In [1]:
# Установка необходимых библиотек
!pip install torch torchvision torchaudio
!pip install transformers
!pip install rouge-score
!pip install scikit-learn
!pip install tqdm
!pip install pandas
!pip install numpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 161.2 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.5/74.5 MB 185.7 kB/s eta 0:00:0000:0100:11
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 808.1/808.1 kB 177.7 kB/s eta 0:00:00a 0:00:02
  Attempting uninstall: torch
    Found existing installation: torch 2.7.1
    Uninstalling torch-2.7.1:
      Successfully uninstalled torch-2.7.1

[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip

In [2]:
import os
import torch
import pandas as pd
from tqdm import tqdm
import numpy as np

# Устройство
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Используется устройство: {DEVICE}")

Используется устройство: cpu


In [3]:
# Выполняем предварительную обработку данных
# Импортируем функции из data_utils.py

import sys
sys.path.append("src")

from src.data_utils import main as preprocess_data

# Запуск обработки данных
preprocess_data()

Загрузка и очистка данных...


/Users/adam/projects/auto_complete_model/src/data_utils.py:32: DtypeWarning: Columns (1,2,3,4,6,7,8,10,12,16,20,24,29,31) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(RAW_DATA_PATH)


Обработано 1048575 текстов.
Токенизация текстов...
Обработанный датасет сохранён в data/dataset_processed.csv
Разбиение на train/val/test...
Размер обучающей выборки: 838860
Размер валидационной выборки: 104857
Размер тестовой выборки: 104858
Разбиение завершено.


In [4]:
# Просмотр обработанных данных
df_train = pd.read_csv("data/train.csv")
print("Примеры обучающих текстов:")
print(df_train.head())
print(f"\nРазмер обучающей выборки: {len(df_train)}")

Примеры обучающих текстов:
                                                text  \
0  the switcher died in the middle of the service...   
1  just accidentally ruined dollhouse for jvthor ...   
2                             wesaysummerrr you have   
3  epiphanygirl since ur bham al show is sold owt...   
4        jefferymavers you havent updated in forever   

                                              tokens  
0  ['the', 'switcher', 'died', 'in', 'the', 'midd...  
1  ['just', 'accidentally', 'ruined', 'dollhouse'...  
2                   ['wesaysummerrr', 'you', 'have']  
3  ['epiphanygirl', 'since', 'ur', 'bham', 'al', ...  
4  ['jefferymavers', 'you', 'havent', 'updated', ...  

Размер обучающей выборки: 838860


In [ ]:
# Запуск обучения LSTM
from src.lstm_train import train_model

train_model()

In [ ]:
# Оценка LSTM с вычислением ROUGE
from src.eval_lstm import evaluate_on_dataset
from src.lstm_model import LSTMModel
import pickle

# Загрузка модели
with open("models/vocab.pkl", "rb") as f:
    vocab = pickle.load(f)
vocab_size = len(vocab)
pad_idx = vocab["<PAD>"]

model = LSTMModel(
    vocab_size=vocab_size,
    embedding_dim=128,
    hidden_dim=256,
    num_layers=2,
    dropout=0.3,
    pad_idx=pad_idx
).to(DEVICE)

model.load("models/lstm_model.pth", device=DEVICE)

# Оценка на валидации
lstm_rouge = evaluate_on_dataset(
    model=model,
    split="val",
    batch_size=64,
    max_samples=500,
    device=DEVICE
)

In [ ]:
# Оценка LSTM с вычислением ROUGE
from src.eval_lstm import evaluate_on_dataset
from src.lstm_model import LSTMModel

# Загрузка модели
with open("models/vocab.pkl", "rb") as f:
    vocab = pickle.load(f)
    
vocab_size = len(vocab)
pad_idx = vocab["<PAD>"]

model = LSTMModel(
    vocab_size=vocab_size,
    embedding_dim=128,
    hidden_dim=256,
    num_layers=2,
    dropout=0.3,
    pad_idx=pad_idx
).to(DEVICE)

model.load("models/lstm_model.pth", device=DEVICE)

# Оценка на валидации
lstm_rouge = evaluate_on_dataset(
    model=model,
    split="val",
    batch_size=64,
    max_samples=500,
    device=DEVICE
)

In [ ]:
# Оценка предобученной модели DistilGPT-2
from src.eval_transformer_pipeline import main as evaluate_transformer

transformer_rouge = evaluate_transformer()

In [ ]:
# Сравнение метрик
results = {
    "Модель": ["LSTM", "DistilGPT-2"],
    "ROUGE-1": [lstm_rouge['rouge1'], transformer_rouge['rouge1']],
    "ROUGE-2": [lstm_rouge['rouge2'], transformer_rouge['rouge2']],
    "ROUGE-L": [lstm_rouge['rougeL'], transformer_rouge['rougeL']],
}

results_df = pd.DataFrame(results)
print("Сравнение моделей по ROUGE-метрикам:")
results_df.round(4)

In [ ]:
# Примеры от LSTM
from src.lstm_model import LSTMModel
from src.eval_lstm import generate_examples

reverse_vocab = {idx: token for token, idx in vocab.items()}

print("=== LSTM: Примеры автодополнения ===")
generate_examples(
    model=model,
    sample_texts=["hello how are", "the weather today", "i want to tell", "in the bank there was"],
    vocab=vocab,
    reverse_vocab=reverse_vocab,
    device=DEVICE,
    max_length=10
)

# Примеры от DistilGPT-2
from src.eval_transformer_pipeline import generate_transformer_examples
from transformers import GPT2LMHeadModel, GPT2Tokenizer

transformer_model, tokenizer = GPT2LMHeadModel.from_pretrained("distilgpt2"), GPT2Tokenizer.from_pretrained("distilgpt2")
transformer_model.config.pad_token_id = tokenizer.pad_token_id
transformer_model = transformer_model.to(DEVICE).eval()

print("=== DistilGPT-2: Примеры автодополнения ===")
generate_transformer_examples(
    model=transformer_model,
    tokenizer=tokenizer,
    sample_texts=["hello how are", "the weather today", "i want to tell", "in the bank there was"],
    max_gen_length=10
)

### 📊 Выводы

1. **Качество генерации**:
   - **DistilGPT-2** демонстрирует значительно более естественные и контекстуально корректные продолжения.
   - **LSTM** часто генерирует шаблонные или бессмысленные фразы, особенно при длинных продолжениях.

2. **ROUGE-метрики**:
   - DistilGPT-2 превосходит LSTM по всем метрикам (ROUGE-1, ROUGE-2, ROUGE-L), что говорит о лучшем соответствии с эталонными продолжениями.

3. **Вычислительные требования**:
   - LSTM: легковесная, быстро обучается, подходит для сред с ограниченными ресурсами.
   - DistilGPT-2: требует больше памяти и времени на инференс, но не требует обучения — только инференс.

4. **Поддержка и масштабируемость**:
   - DistilGPT-2 обучалась на огромном корпусе, что даёт ей преимущество в понимании языка.
   - LSTM требует большого качественного датасета и тонкой настройки.

---

### ✅ Рекомендации разработчикам

**Использовать DistilGPT-2 (или аналогичную предобученную модель)**, если:
- Качество автодополнения — приоритет.
- Есть возможность выделить ресурсы на инференс (GPU или оптимизация через ONNX, DistilBERT и т.п.).
- Не требуется полный контроль над обучением.

**Остановиться на LSTM**, если:
- Жёсткие ограничения по памяти и задержкам (например, мобильные устройства).
- Данные сильно доменные и нет возможности дообучать трансформер.
- Требуется полная автономия и контроль над моделью.

---

### 🔄 Дальнейшие улучшения
- Дообучить DistilGPT-2 на внутренних текстах банка (fine-tuning).
- Оптимизировать инференс: квантование, ONNX, кэширование.
- Реализовать ранжирование нескольких кандидатов.
- Добавить фильтрацию чувствительной информации.

In [ ]:
# Сохранение результатов
results_df.to_csv("results/comparison_results.csv", index=False)
print("Итоговые результаты сохранены в results/comparison_results.csv")